In [ ]:
#合并 chinese-alpaca-plus-lora-7b
!cd Chinese-LLaMA-Alpaca && python scripts/merge_llama_with_chinese_lora.py \
    --base_model /data/llama/llama_7b_hf \
    --lora_model /data/llama/chinese-llama-plus-lora-7b,/data/llama/chinese-alpaca-plus-lora-7b \
    --output_type huggingface \
    --output_dir /data/llama/merge_chinese_alpaca_plus_lora_7b

In [ ]:
#准备精调数据
!mkdir Chinese-LLaMA-Alpaca/sft_data
!cp Chinese-LLaMA-Alpaca/data/alpaca_data_zh_51k.json Chinese-LLaMA-Alpaca/sft_data

In [ ]:
#使用合并后的模型作为基模型训练
#model_name_or_path         合并后的完整模型
#tokenizer_name_or_path     扩充后的词表路径,这里使用合并后模型的词表也可以
#peft_path                  训练好未合并的权重模型,比如chinese-llama-plus-lora-7b,chinese-alpaca-plus-lora-7b或者自己经过预训练得到的权重模型

!cd Chinese-LLaMA-Alpaca/scripts/training && torchrun --nnodes 1 --nproc_per_node 1 run_clm_sft_with_peft.py \
    --deepspeed ds_zero2_no_offload.json \
    --model_name_or_path /data/llama/merge_nst_pt_chinese_alpaca_plus_lora_7b \
    --tokenizer_name_or_path /data/llama/merge_nst_pt_chinese_alpaca_plus_lora_7b \
    --dataset_dir /data/wenbo/Chinese-LLaMA-Alpaca/sft_data \
    --validation_split_percentage 0.001 \
    --per_device_train_batch_size 1 \
    --do_train \
    --fp16 \
    --seed $RANDOM \
    --max_steps 100 \
    --lr_scheduler_type cosine \
    --learning_rate 1e-4 \
    --warmup_ratio 0.03 \
    --weight_decay 0 \
    --logging_strategy steps \
    --logging_steps 10 \
    --save_strategy steps \
    --save_total_limit 3 \
    --save_steps 50 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 8 \
    --max_seq_length 512 \
    --output_dir /data/wenbo/Chinese-LLaMA-Alpaca/output/sft_model \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --torch_dtype float16 \
    --peft_path /data/wenbo/Chinese-LLaMA-Alpaca/output/pt_model/pt_lora_model \
    --gradient_checkpointing \
    --ddp_find_unused_parameters False

#将训练好的权重与基模型合并(这里的基础模型指的是merge_chinese_alpaca_plus_lora_7b合并后的模型)
!cd Chinese-LLaMA-Alpaca && python scripts/merge_llama_with_chinese_lora.py \
    --base_model /data/llama/merge_chinese_alpaca_plus_lora_7b \
    --lora_model /data/wenbo/Chinese-LLaMA-Alpaca/output/sft_model/sft_lora_model \
    --output_type huggingface \
    --output_dir /data/llama/nst-sft-chinese-alpaca-plus-lora-7b

#启动原训练合并后的hf模型演示
!python scripts/inference/inference_hf.py --base_model /data/llama/nst-sft-chinese-alpaca-plus-lora-7b --with_prompt --interactive --gpus 0